In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import pickle
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test
import matplotlib.pyplot as plt
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot as upset
import missingno as msno
from pyod.models.lunar import LUNAR
from matplotlib_venn import venn2, venn2_circles
from glob import glob
from hydra import compose, initialize
from omegaconf import OmegaConf
import omegaconf
import os

# 0. Setup

In [ ]:
path_dataset = "E:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
path_load = f"{path_dataset}/data/covid/treatment"
path_save = f"{path_dataset}/special/041_covid_treatment"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

# 1. Empty features

In [ ]:
path_local = f"001_empty_features"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)

df = pd.read_excel(f"{path_load}/data_0.xlsx", index_col="patient_id")

series_n_nan = df.isna().sum()
df_nan = pd.DataFrame({'n_nans': series_n_nan.values}, index = series_n_nan.index)
df_nan.sort_values([f"n_nans"], ascending=[False], inplace=True)

hist_min = df_nan.loc[:, f"n_nans"].min()
hist_max = df_nan.loc[:, f"n_nans"].max()
hist_width = hist_max - hist_min
hist_n_bins = df_nan.loc[:, f"n_nans"].max()
hist_bin_width = hist_width / hist_n_bins

plt.figure()
sns.set_theme(style='whitegrid')
hist = sns.histplot(
    data=df_nan,
    x=f"n_nans",
    bins=hist_n_bins,
    binrange=(hist_min, hist_max),
    binwidth=hist_bin_width,
    discrete=True,
    edgecolor='k',
    linewidth=1
)
hist.set_xlabel("Number of missing values")
hist.set_ylabel("Number of features")
hist.set_title(f"Total features: {df.shape[1]}\nTotal samples: {df.shape[0]}")
plt.savefig(f"{path_save}/{path_local}/hist_n_nans.png", bbox_inches='tight')
plt.savefig(f"{path_save}/{path_local}/hist_n_nans.pdf", bbox_inches='tight')
plt.clf()

lim_exclude = df.shape[0] * 0.9
feats_exclude = df_nan.index[df_nan["n_nans"] > lim_exclude].values

df.drop(feats_exclude, axis=1, inplace=True)
df['n_nans'] = df.isnull().sum(axis=1)

df.to_excel(f"{path_load}/data_1.xlsx")

## 2. Initial analysis

In [ ]:
path_local = f"002_initial_analysis"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)

df = pd.read_excel(f"{path_load}/data_1.xlsx", index_col="patient_id")

series_n_nan = df.isna().sum()
df_nan = pd.DataFrame({'n_nans': series_n_nan.values}, index = series_n_nan.index)
df_nan.sort_values([f"n_nans"], ascending=[False], inplace=True)

hist_min = df_nan.loc[:, f"n_nans"].min()
hist_max = df_nan.loc[:, f"n_nans"].max()
hist_width = hist_max - hist_min
hist_n_bins = df_nan.loc[:, f"n_nans"].max()
hist_bin_width = hist_width / hist_n_bins

plt.figure()
sns.set_theme(style='whitegrid')
hist = sns.histplot(
    data=df_nan,
    x=f"n_nans",
    bins=hist_n_bins,
    binrange=(hist_min, hist_max),
    binwidth=hist_bin_width,
    discrete=True,
    edgecolor='k',
    linewidth=1
)
hist.set_xlabel("Number of missing values")
hist.set_ylabel("Number of features")
hist.set_title(f"Total features: {df.shape[1]}\nTotal samples: {df.shape[0]}")
plt.savefig(f"{path_save}/{path_local}/hist_n_nans.png", bbox_inches='tight')
plt.savefig(f"{path_save}/{path_local}/hist_n_nans.pdf", bbox_inches='tight')
plt.clf()

msno.bar(
    df=df,
    label_rotation=90
)
plt.savefig(f"{path_save}/{path_local}/msno_bar.png", bbox_inches='tight')
plt.savefig(f"{path_save}/{path_local}/msno_bar.pdf", bbox_inches='tight')
plt.clf()

msno.matrix(
    df=df,
    label_rotation=90
)
plt.savefig(f"{path_save}/{path_local}/msno_mtx.png", bbox_inches='tight')
plt.savefig(f"{path_save}/{path_local}/msno_mtx.pdf", bbox_inches='tight')
plt.clf()

msno.matrix(
    df=df.sort_values([f"n_nans"], ascending=[False]),
    label_rotation=90
)
plt.savefig(f"{path_save}/{path_local}/msno_mtx_sorted.png", bbox_inches='tight')
plt.savefig(f"{path_save}/{path_local}/msno_mtx_sorted.pdf", bbox_inches='tight')
plt.clf()

## 3. Features

In [36]:
path_local = f"003_features"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)

df = pd.read_excel(f"{path_load}/data_1.xlsx", index_col="patient_id")